# 05 - resumo de venda/faturamento - fechamento anual


In [ ]:
''' mes-ano / vendas / faturado 
total / .../ '''

In [ ]:
# -- 1 - defines as bases de dados e trata certas colunas
df_nfs = df_nfs_masquerade[['Vl Total Nota','Dt. emissão']]
df_pedidos = df_pedidos_masquerade[['Vl venda','Dt. emissão']]

# -- 2 - pega os anos
anos = [2024, 2025]

# -- 3 - filtra com base nos anos e deixa penas o mes e o ano 
df_pedidos = df_pedidos[ # filtro
            (df_pedidos['Dt. emissão'].dt.year == anos[0]) |
            (df_pedidos['Dt. emissão'].dt.year == anos[1]) 
            ] 

df_nfs = df_nfs[ # filtro
            (df_nfs['Dt. emissão'].dt.year == anos[0]) |
            (df_nfs['Dt. emissão'].dt.year == anos[1]) 
            ] 

# -- 4 - trandorma em mes-ano
df_pedidos['Dt. emissão'] = df_pedidos['Dt. emissão'].dt.to_period('M')
df_nfs['Dt. emissão'] = df_nfs['Dt. emissão'].dt.to_period('M')

# -- 5 - Agrupa por mês e soma os valores
vendas_mensal = df_pedidos.groupby('Dt. emissão')['Vl venda'].sum().reset_index()
faturado_mensal = df_nfs.groupby('Dt. emissão')['Vl Total Nota'].sum().reset_index()

# -- 7 -  mergga os dois DataFrames pela coluna 'Dt. emissão'
df_raw_mes_ano = pd.merge(
    vendas_mensal,
    faturado_mensal,
    on='Dt. emissão',
    how='outer'  # inclui meses que têm apenas vendas ou apenas faturamento
).fillna(0)

# -- 8 - Ordena pelo mês-ano corretamente
df_raw_mes_ano = df_raw_mes_ano.sort_values('Dt. emissão')

# -- 9 - mud o nome ds colunas
df_raw_mes_ano.columns = ['mes-ano', 'venda', 'faturado']

total_2024_faturado = df_raw_mes_ano['faturado'].sum()
total_2024_venda = df_raw_mes_ano['venda'].sum()

print(f'2024 - total:\n faturado | venda \n {total_2024_faturado} | {total_2024_venda} ')
df_raw_mes_ano